In [3]:
from glove import Glove
import gensim
import numpy as np

In [1]:
txt_file = "/home/ubuntu/summarization_query_oriented/data/wikipedia/txt/td_qfs_rank_1_all.txt"
with open(txt_file, 'r') as f:
    text = f.read()

In [4]:

word_embedder_folder = "/home/ubuntu/summarization_query_oriented/nn_models/word_vectorization_models/glove/"
word_embedder = "rank1_alltxt_size_400_epoch_30.glove"

glove = Glove.load(word_embedder_folder+word_embedder) 


In [5]:
type(text)

str

In [6]:
all_sentences = text.decode("utf-8").split(".")

In [16]:
len(all_sentences)

206714

In [7]:
for sentence in all_sentences[:2]:
    print gensim.utils.simple_preprocess(sentence, deacc=False, min_len=2, max_len=15)

[u'bauxite', u'pneumoconiosis', u'also', u'known', u'as', u'shaver', u'disease', u'corundum', u'smelter', u'lung', u'bauxite', u'lung', u'or', u'bauxite', u'smelters', u'disease', u'is', u'progressive', u'form', u'of', u'pneumoconiosis', u'caused', u'by', u'exposure', u'to', u'bauxite', u'fumes', u'which', u'contain', u'aluminium', u'and', u'silica', u'particulates']
[u'it', u'is', u'typically', u'seen', u'in', u'workers', u'involved', u'in', u'the', u'smelting', u'of', u'bauxite', u'to', u'produce', u'corundum']


In [8]:
def one_hot_vector(word ,corpus):
    """
    given a corpus of (key,value) = (word,index) , this function return a one hot vector corresponding the input word
    """
    size = len(corpus.values())
    y = np.zeros(size)
    y[corpus[word]] = 1.0
    return y


In [9]:
def bidirectional_batch_generator(text, glove_model):
    """
    Generator of bidirectional batchs to train on a regular LSTM model
    It yields :
    - X a batch representing a single sentence, therefore the size of the batch vary between each batch
    - y a batch of one hot vectors representing the next word to find
    
    """
    corpus = glove_model.dictionary
    corpus_size = len(corpus.keys())
    all_sentences = text.decode("utf-8").split(".")
    for sentence in all_sentences :
        
        word_list = gensim.utils.simple_preprocess(sentence, deacc=False, min_len=2, max_len=15)
        corpus[u'SENTENCE_START'] = corpus_size
        corpus[u'SENTENCE_END'] = corpus_size + 1
        
        word_list.insert(0,u'SENTENCE_START')
        word_list.append(u'SENTENCE_END')
        
        for direction in [True,False] : 
            X = []
            y = []

            if not direction : word_list = word_list[::-1]

            for i in range(len(word_list)-1):
                if corpus[word_list[i]] <corpus_size:
                    X.append(glove_model.word_vectors[corpus[word_list[i]]])
                else :
                    # case sentence_start sentence_end
                    X.append(np.zeros(glove.no_components)) 

                y.append(one_hot_vector(word_list[i+1],corpus))    
                
            X = np.asarray(X)
            y = np.asarray(y)
            yield X,y

In [10]:
generat = bidirectional_batch_generator(text, glove)

In [11]:
X, y = generat.next()

In [12]:
X.shape

(34, 400)

In [13]:
for i in range(len(y)):
    word_idx = np.arange(len(glove.dictionary.keys())) [y[i,:]==1.0]
    for key in glove.dictionary.keys():
        if glove.dictionary[key] == word_idx :
            print key

bauxite
pneumoconiosis
also
known
as
shaver
disease
corundum
smelter
lung
bauxite
lung
or
bauxite
smelters
disease
is
progressive
form
of
pneumoconiosis
caused
by
exposure
to
bauxite
fumes
which
contain
aluminium
and
silica
particulates
SENTENCE_END


In [14]:
type(glove.word_vectors)

numpy.ndarray

In [15]:
glove.no_components

400